In [6]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

In [9]:
nevts = 20145
nfoscevts = 8784618

In [11]:
with open("data/observed_events.dat") as f:
    data = pd.read_table(f, sep=' ', header=None, usecols=[0,1],names=['energy','zenith'],skiprows=12,lineterminator='\n')
    
muonenergyproxy = data.as_matrix(['energy'])
recomuonzenith = data.as_matrix(['zenith'])

In [12]:
# Now, the nominal signal
with open("data/NuFSGenMC_nominal.dat") as f:
    data = pd.read_table(f, sep=' ', header=None, usecols=[1,2,5,6,7],names=['energy','zenith','mc_weight','pion_flux','kaon_flux'],skiprows=11,lineterminator='\n')

nom_muonenergyproxy = data.as_matrix(['energy'])
nom_recomuonzenith = data.as_matrix(['zenith'])
nom_weight = (data.as_matrix(['pion_flux'])+data.as_matrix(['kaon_flux'])) * data.as_matrix(['mc_weight'])

In [ ]:

	// Fill up a histo
	//I want evenly spaced log bins
	float emin = 400;	float emax = 20000;
	int nbins = 28;
	float bins[nbins+1];
	for(int i = 0; i < nbins+1; i++){
		bins[i] = pow(10,log10(emin) + i*log10(emax/emin)/nbins);
	}

	TH1F *h = new TH1F("h","Reconstructed Events;Energy/GeV",nbins,bins);
	TH1F *nom = new TH1F("nom","nom",nbins,bins);
	for(int i = 0; i < nEvts; i++){
		h->Fill(muonEnergyProxy[i]);
	}
	for(int i = 0; i < nFOscEvts; i++){
		nom->Fill(nom_muonEnergyProxy[i],nom_weight[i]);
	}

	// Now, we've gotta normalize the bins of the nominal distribution to match the signal.
	int datact = 0;	int nomct = 0;
	for(int b = 1; b < nbins+1; b++){
		datact += h->GetBinContent(b);
		nomct += nom->GetBinContent(b);
	}
	for(int i = 1; i < nbins+1; i++){
		nom->SetBinContent(i,nom->GetBinContent(i)*datact/nomct);
	}

	// Draw the histo to check that it all looks right.
	TCanvas *c1 = new TCanvas("c1");
	gStyle->SetFillColor(0);
  	gStyle->SetPadLeftMargin(0.15);
  	gStyle->SetPadBottomMargin(0.15);
  	gStyle->SetOptFit(1);
  	gStyle->SetOptTitle(0);
  	gStyle->SetTitleSize(0.05);
	gStyle->SetHatchesLineWidth(2);

	h->GetXaxis()->SetTitleOffset(1.1);
	h->GetYaxis()->SetTitleOffset(.8);
	h->GetXaxis()->SetTitleFont(62);
	h->GetYaxis()->SetTitleFont(62);
	h->GetYaxis()->CenterTitle();
	h->GetXaxis()->CenterTitle();
	h->GetXaxis()->SetTitleSize(0.04);
	h->GetXaxis()->SetLabelSize(0.04);
	h->GetXaxis()->SetLabelOffset(0.001);
	h->GetYaxis()->SetTitleSize(0.04);
	h->GetYaxis()->SetLabelSize(0.04);
	h->SetStats(kFALSE);
	h->SetMarkerColor(2);
   	h->SetMarkerStyle(4);

	c1->SetLogy();
	c1->SetLogx();

	h->Draw("P");
	nom->Draw("same");
	c1->Print("test.png");
